In [1]:
import intake

from access_intake_utils.chunking import validate_chunkspec


In [2]:
# Let's look at the documentation for validate_chunkspec
validate_chunkspec?

Signature:
validate_chunkspec(
    dataset: str | pathlib.Path | collections.abc.Iterable[str | pathlib.Path] | xarray.core.dataset.Dataset | xarray.core.dataarray.DataArray | intake_esm.core.esm_datastore,
    chunkspec: dict[str, typing.Any],
    varnames: str | list[str] | None = None,
    validate_mode: Literal['single', 'bookend', 'sample', 'all'] = 'single',
    sample_size: int = 10,
) -> dict | dict[pathlib.Path, dict]
Docstring:
Validate the chunk sizes for a given variable name.


This function attempts to validate that user specified chunks are compatible
with chunking on disk, by ensuring that chunk sizes are integer multiples of
the chunk sizes on disk.

Parameters
----------
dataset: str | Path | Iterable[str | Path] | Dataset | DataArray | esm_datastore
    A very general object that can be used to open a single (or many) netCDF
    files.
chunkspec : dict[str, Any]
    The chunk specification dictionary, used to tell xarray how to chunk the data.
varnames : str | list[s

In [3]:
# Let's load up an esm_datastore to see how we can use this
catalog = intake.cat.access_nri
expt_ds = catalog['01deg_jra55_ryf_Control']
expt_ds = expt_ds.search(file_id='ocean_month',variable='temp') # Subset down to a single dataset & variable
expt_ds

,unique
filename,1
file_id,1
path,14
filename_timestamp,0
frequency,1
start_date,14
end_date,14
variable,56
variable_long_name,52
variable_standard_name,23


In [4]:
# Let's look at our datastore
expt_ds.to_dask(xarray_open_kwargs={'decode_timedelta' : False})

<xarray.Dataset> Size: 122GB
Dimensions:   (time: 42, st_ocean: 75, yt_ocean: 2700, xt_ocean: 3600)
Coordinates:
  * xt_ocean  (xt_ocean) float64 29kB -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
  * yt_ocean  (yt_ocean) float64 22kB -81.11 -81.07 -81.02 ... 89.89 89.94 89.98
  * st_ocean  (st_ocean) float64 600B 0.5413 1.681 2.94 ... 5.511e+03 5.709e+03
  * time      (time) object 336B 2145-01-16 12:00:00 ... 2148-06-16 00:00:00
Data variables:
    temp      (time, st_ocean, yt_ocean, xt_ocean) float32 122GB dask.array<chunksize=(1, 19, 135, 180), meta=np.ndarray>
Attributes: (12/16)
    filename:                                 ocean_month.nc
    title:                                    ACCESS-OM2-01
    grid_type:                                mosaic
    grid_tile:                                1
    intake_esm_vars:                          ['temp']
    intake_esm_attrs:filename:                ocean_month.nc
    ...                                       ...
    intake_esm_attrs:variable_standard_name:  ,,,,,,,,,,,,,,,,,,sea_water_con...
    intake_esm_attrs:variable_cell_methods:   ,,,,,,,,,,,,,,,,,,time: mean,ti...
    intake_esm_attrs:variable_units:          degrees_E,degrees_N,meters,mete...
    intake_esm_attrs:realm:                   ocean
    intake_esm_attrs:_data_format_:           netcdf
    intake_esm_dataset_key:                   ocean_month.1mon

## Okay. Lets see what happens if we try to increase our chunk size to some random prime numbers (these are *very unlikely* to be integer multiples of the disk chunking)

In [16]:
chunks = {'time' : 37, 'st_ocean' : 61, 'yt_ocean': 467, 'xt_ocean': 277}

In [17]:
validate_chunkspec(expt_ds, chunks,varnames = 'temp')

/jobfs/140304826.gadi-pbs/ipykernel_2541108/2116134971.py:1: ChunkingWarning: Specified chunks are not integer multiples of the disk chunks. Returning suggested chunks as a dictionary.
  validate_chunkspec(expt_ds, chunks,varnames = 'temp')


{'time': 37, 'st_ocean': 57, 'yt_ocean': 405, 'xt_ocean': 360}

In [10]:
expt_ds.df.path.head(1)

0    /g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55_ryf_Control/output976/ocean/ocean_month.nc
Name: path, dtype: object

In [15]:
# Let's look at the chunks on disk with ncdump -hs (and filter other variables out'
!ncdump -hs /g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55_ryf_Control/output976/ocean/ocean_month.nc   | head -160 | tail -15

	float temp(time, st_ocean, yt_ocean, xt_ocean) ;
		temp:long_name = "Conservative temperature" ;
		temp:units = "K" ;
		temp:valid_range = -10.f, 500.f ;
		temp:missing_value = -1.e+20f ;
		temp:_FillValue = -1.e+20f ;
		temp:cell_methods = "time: mean" ;
		temp:time_avg_info = "average_T1,average_T2,average_DT" ;
		temp:coordinates = "geolon_t geolat_t" ;
		temp:standard_name = "sea_water_conservative_temperature" ;
		temp:_Storage = "chunked" ;
		temp:_ChunkSizes = 1, 19, 135, 180 ;
		temp:_Shuffle = "true" ;
		temp:_DeflateLevel = 5 ;
		temp:_Endianness = "little" ;


So temperature has disk chunks `{'time': 1 , 'st_ocean' : 19, 'yt_ocean' : 135, 'xt_ocean' : 180}`

`validate_chunkspec` has returned `{'time': 37, 'st_ocean': 57, 'yt_ocean': 405, 'xt_ocean': 360}` which is
`{'time': 37 * 1, 'st_ocean': 19 * 3, 'yt_ocean': 135 * 3, 'xt_ocean' : 180 * 2}`, which is the closest we can get to the original chunks, whilst respecting the disk chunking. We can then load our dataset with this:

In [18]:
optimised_chunks = validate_chunkspec(expt_ds, chunks,varnames = 'temp')
expt_ds.to_dask(xarray_open_kwargs={'decode_timedelta' : False, 'chunks' : optimised_chunks})

/jobfs/140304826.gadi-pbs/ipykernel_2541108/677627901.py:1: ChunkingWarning: Specified chunks are not integer multiples of the disk chunks. Returning suggested chunks as a dictionary.
  optimised_chunks = validate_chunkspec(expt_ds, chunks,varnames = 'temp')
/g/data/xp65/public/apps/med_conda/envs/analysis3-25.02/lib/python3.11/site-packages/intake_esm/source.py:82: UserWarning: The specified chunks separate the stored chunks along dimension "yt_ocean" starting at index 405. This could degrade performance. Instead, consider rechunking after loading.
  ds = xr.open_dataset(url, **xarray_open_kwargs)
/g/data/xp65/public/apps/med_conda/envs/analysis3-25.02/lib/python3.11/site-packages/intake_esm/source.py:82: UserWarning: The specified chunks separate the stored chunks along dimension "xt_ocean" starting at index 360. This could degrade performance. Instead, consider rechunking after loading.
  ds = xr.open_dataset(url, **xarray_open_kwargs)


<xarray.Dataset> Size: 122GB
Dimensions:   (time: 42, st_ocean: 75, yt_ocean: 2700, xt_ocean: 3600)
Coordinates:
  * xt_ocean  (xt_ocean) float64 29kB -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
  * yt_ocean  (yt_ocean) float64 22kB -81.11 -81.07 -81.02 ... 89.89 89.94 89.98
  * st_ocean  (st_ocean) float64 600B 0.5413 1.681 2.94 ... 5.511e+03 5.709e+03
  * time      (time) object 336B 2145-01-16 12:00:00 ... 2148-06-16 00:00:00
Data variables:
    temp      (time, st_ocean, yt_ocean, xt_ocean) float32 122GB dask.array<chunksize=(3, 57, 405, 360), meta=np.ndarray>
Attributes: (12/16)
    filename:                                 ocean_month.nc
    title:                                    ACCESS-OM2-01
    grid_type:                                mosaic
    grid_tile:                                1
    intake_esm_vars:                          ['temp']
    intake_esm_attrs:filename:                ocean_month.nc
    ...                                       ...
    intake_esm_attrs:variable_standard_name:  ,,,,,,,,,,,,,,,,,,sea_water_con...
    intake_esm_attrs:variable_cell_methods:   ,,,,,,,,,,,,,,,,,,time: mean,ti...
    intake_esm_attrs:variable_units:          degrees_E,degrees_N,meters,mete...
    intake_esm_attrs:realm:                   ocean
    intake_esm_attrs:_data_format_:           netcdf
    intake_esm_dataset_key:                   ocean_month.1mon

# What if I don't want to use intake?

- We can pass a list of files and/or paths:


In [20]:
paths_as_strings = expt_ds.df.path.tolist()
paths_as_strings

['/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55_ryf_Control/output976/ocean/ocean_month.nc',
 '/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55_ryf_Control/output977/ocean/ocean_month.nc',
 '/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55_ryf_Control/output978/ocean/ocean_month.nc',
 '/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55_ryf_Control/output979/ocean/ocean_month.nc',
 '/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55_ryf_Control/output980/ocean/ocean_month.nc',
 '/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55_ryf_Control/output981/ocean/ocean_month.nc',
 '/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55_ryf_Control/output982/ocean/ocean_month.nc',
 '/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55_ryf_Control/output983/ocean/ocean_month.nc',
 '/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55_ryf_Control/output984/ocean/ocean_month.nc',
 '/g/data/cj50/access-om2/ra

In [21]:
validate_chunkspec(paths_as_strings, chunks,varnames = 'temp')

/jobfs/140304826.gadi-pbs/ipykernel_2541108/3620575421.py:1: ChunkingWarning: Specified chunks are not integer multiples of the disk chunks. Returning suggested chunks as a dictionary.
  validate_chunkspec(paths_as_strings, chunks,varnames = 'temp')


{'time': 37, 'st_ocean': 57, 'yt_ocean': 405, 'xt_ocean': 360}

In [25]:
from pathlib import Path

paths = [Path(pathstr) for pathstr in paths_as_strings]
paths

[PosixPath('/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55_ryf_Control/output976/ocean/ocean_month.nc'),
 PosixPath('/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55_ryf_Control/output977/ocean/ocean_month.nc'),
 PosixPath('/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55_ryf_Control/output978/ocean/ocean_month.nc'),
 PosixPath('/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55_ryf_Control/output979/ocean/ocean_month.nc'),
 PosixPath('/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55_ryf_Control/output980/ocean/ocean_month.nc'),
 PosixPath('/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55_ryf_Control/output981/ocean/ocean_month.nc'),
 PosixPath('/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55_ryf_Control/output982/ocean/ocean_month.nc'),
 PosixPath('/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55_ryf_Control/output983/ocean/ocean_month.nc'),
 PosixPath('/g/data/cj50/access-om2/raw-output/a

In [26]:
validate_chunkspec(paths, chunks,varnames = 'temp')

/jobfs/140304826.gadi-pbs/ipykernel_2541108/883754652.py:1: ChunkingWarning: Specified chunks are not integer multiples of the disk chunks. Returning suggested chunks as a dictionary.
  validate_chunkspec(paths, chunks,varnames = 'temp')


{'time': 37, 'st_ocean': 57, 'yt_ocean': 405, 'xt_ocean': 360}

- Or, in some instances, an xarray dataset - but only if the dataset contains the file handles (this isn't guaranteed, so preferably use the other methods). In the example below, this won't work - but it might for others. This example will be updated as suppot for xarray detasets becomes more robust.

In [29]:
import xarray as xr

ds = xr.open_mfdataset(paths, decode_timedelta = False)

validate_chunkspec(ds, chunks,varnames = 'temp')

ValueError:  Dataset/DataArray does contain source attribute describing file path(s). Please provide a dataset with a source attribute, an esm_datastore, or a list of file paths.